In [1]:
!pip install optuna

     |████████████████████████████████| 308 kB 7.5 MB/s 
     |████████████████████████████████| 210 kB 68.6 MB/s 
     |████████████████████████████████| 81 kB 12.9 MB/s 
     |████████████████████████████████| 78 kB 9.4 MB/s 
     |████████████████████████████████| 49 kB 8.3 MB/s 
     |████████████████████████████████| 146 kB 65.9 MB/s 
     |████████████████████████████████| 113 kB 74.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=1a2f4e61f0565fae88a72ebb10360b4a84e7a160e3d962d2593bbd17fa2f082c
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [2]:
import os
import argparse
import numpy as np
import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.BatchNorm1d(120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def create_dataloader(batch_size=64, valid_size=0.2, DIR='D:/datasets/'):
    # convert data to torch.FloatTensor
    transform = transforms.ToTensor()

    # choose the training and test datasets
    train_data = datasets.FashionMNIST(root=DIR,
                                train=True,
                                download=True,
                                transform=transform)
    test_data = datasets.FashionMNIST(root=DIR,
                               train=False,
                               download=False,
                               transform=transform)

    # obtain training indices that will be used for validation
    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    # define samplers for obtaining training and validation batches
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    # load training data in batches
    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=train_sampler,
                                               num_workers=0)

    # load validation data in batches
    valid_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=valid_sampler,
                                               num_workers=0)

    # load test data in batches
    test_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size=batch_size,
                                              num_workers=0)

    return train_loader, valid_loader, test_loader


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    train_accuracy = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        acc = pred.eq(target.view_as(pred)).sum().item() / args.batch_size
        train_accuracy += acc
        loss = args.criterion(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), acc,
            ))


def validate(args, model, device, val_loader):
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += args.criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            val_acc += pred.eq(target.view_as(pred)).sum().item() / args.batch_size
    val_loss /= len(val_loader)
    val_acc /= len(val_loader)
    return val_loss, val_acc


def get_args():
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1024, metavar='N',
                        help='input batch size for testing (default: 1024)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=500, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--criterion', default=nn.CrossEntropyLoss(),
                        help='loss function of training')
    parser.add_argument('--model-path', type=str, default='./models/LeNet_tune.pt',
                        help='directory to save model')
    parser.add_argument('--optimizer-name', type=str, default='Adam',
                        help='name of the optimier')
    args = parser.parse_args(args=[])
    return args
args = get_args()


def objective(trial):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    model = LeNet().to(device)

    optimizer_name = args.optimizer_name
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loader, val_loader, _ = create_dataloader()

    for epoch in range(args.epochs):
        train(args, model, device, train_loader, optimizer, epoch)
        _, val_acc = validate(args, model, device, val_loader)
        trial.report(val_acc, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_acc

In [7]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.BatchNorm1d(120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def create_dataloader(batch_size=64, valid_size=0.2, DIR='D:/datasets/'):
    # convert data to torch.FloatTensor
    transform = transforms.ToTensor()

    # choose the training and test datasets
    train_data = datasets.FashionMNIST(root=DIR,
                                train=True,
                                download=False,
                                transform=transform)
    test_data = datasets.FashionMNIST(root=DIR,
                               train=False,
                               download=False,
                               transform=transform)

    # obtain training indices that will be used for validation
    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    # define samplers for obtaining training and validation batches
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    # load training data in batches
    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=train_sampler,
                                               num_workers=0)

    # load validation data in batches
    valid_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=valid_sampler,
                                               num_workers=0)

    # load test data in batches
    test_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size=batch_size,
                                              num_workers=0)

    return train_loader, valid_loader, test_loader


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    train_accuracy = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        acc = pred.eq(target.view_as(pred)).sum().item() / args.batch_size
        train_accuracy += acc
        loss = args.criterion(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), acc,
            ))


def validate(args, model, device, val_loader):
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += args.criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            val_acc += pred.eq(target.view_as(pred)).sum().item() / args.batch_size
    val_loss /= len(val_loader)
    val_acc /= len(val_loader)
    return val_loss, val_acc


def get_args():
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1024, metavar='N',
                        help='input batch size for testing (default: 1024)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=500, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--criterion', default=nn.CrossEntropyLoss(),
                        help='loss function of training')
    parser.add_argument('--model-path', type=str, default='./models/LeNet_tune.pt',
                        help='directory to save model')
    parser.add_argument('--optimizer-name', type=str, default='Adam',
                        help='name of the optimier')
    args = parser.parse_args(args=[])
    return args
args = get_args()


def objective(trial):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print('Using ', device)

    model = LeNet().to(device)

    optimizer_name = args.optimizer_name
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    args.batch_size = batch_size
    train_loader, val_loader, _ = create_dataloader(batch_size=args.batch_size)

    for epoch in range(args.epochs):
        train(args, model, device, train_loader, optimizer, epoch)
        _, val_acc = validate(args, model, device, val_loader)
        trial.report(val_acc, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_acc

In [20]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.BatchNorm1d(120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def create_dataloader(batch_size=64, valid_size=0.2, DIR='D:/datasets/'):
    # convert data to torch.FloatTensor
    transform = transforms.ToTensor()

    # choose the training and test datasets
    train_data = datasets.FashionMNIST(root=DIR,
                                train=True,
                                download=False,
                                transform=transform)
    test_data = datasets.FashionMNIST(root=DIR,
                               train=False,
                               download=False,
                               transform=transform)

    # obtain training indices that will be used for validation
    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    # define samplers for obtaining training and validation batches
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    # load training data in batches
    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=train_sampler,
                                               num_workers=0)

    # load validation data in batches
    valid_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=valid_sampler,
                                               num_workers=0)

    # load test data in batches
    test_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size=batch_size,
                                              num_workers=0)

    return train_loader, valid_loader, test_loader


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    train_accuracy = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        acc = pred.eq(target.view_as(pred)).sum().item() / args.batch_size
        train_accuracy += acc
        loss = args.criterion(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()


def validate(args, model, device, val_loader):
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += args.criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            val_acc += pred.eq(target.view_as(pred)).sum().item() / args.batch_size
    val_loss /= len(val_loader)
    val_acc /= len(val_loader)
    return val_loss, val_acc


def get_args():
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1024, metavar='N',
                        help='input batch size for testing (default: 1024)')
    parser.add_argument('--epochs', type=int, default=5, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--criterion', default=nn.CrossEntropyLoss(),
                        help='loss function of training')
    parser.add_argument('--model-path', type=str, default='./models/LeNet_tune.pt',
                        help='directory to save model')
    parser.add_argument('--optimizer-name', type=str, default='Adam',
                        help='name of the optimier')
    args = parser.parse_args(args=[])
    return args
args = get_args()


def objective(trial):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    model = LeNet().to(device)

    optimizer_name = args.optimizer_name
    init_lr = trial.suggest_float("init_lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=init_lr)

    gamma = trial.suggest_float("gamma", 0.6, 1.0)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    args.batch_size = batch_size
    train_loader, val_loader, _ = create_dataloader(batch_size=args.batch_size)

    for epoch in range(args.epochs):
        train(args, model, device, train_loader, optimizer, epoch)
        _, val_acc = validate(args, model, device, val_loader)
        print('epoch {} of {}: val_acc: {:.3f}'.format(epoch+1, args.epochs, val_acc))
        scheduler.step()
        trial.report(val_acc, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_acc

In [21]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Using ', device)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-04-20 03:55:03,215] A new study created in memory with name: no-name-f57fb6d4-67c0-4af6-bd4b-472bd3ac8337


Using  cuda
epoch 1 of 5: val_acc: 0.862
epoch 2 of 5: val_acc: 0.883
epoch 3 of 5: val_acc: 0.891
epoch 4 of 5: val_acc: 0.893


[I 2022-04-20 03:55:37,240] Trial 0 finished with value: 0.9005984042553191 and parameters: {'init_lr': 0.0015675612393006848, 'gamma': 0.8632463261350161, 'batch_size': 64}. Best is trial 0 with value: 0.9005984042553191.


epoch 5 of 5: val_acc: 0.901
epoch 1 of 5: val_acc: 0.842
epoch 2 of 5: val_acc: 0.865
epoch 3 of 5: val_acc: 0.873
epoch 4 of 5: val_acc: 0.876


[I 2022-04-20 03:56:04,751] Trial 1 finished with value: 0.8769115691489362 and parameters: {'init_lr': 0.00017235698661844536, 'gamma': 0.6320971021747585, 'batch_size': 128}. Best is trial 0 with value: 0.9005984042553191.


...

epoch 1 of 5: val_acc: 0.832
epoch 1 of 5: val_acc: 0.883
epoch 2 of 5: val_acc: 0.897
epoch 3 of 5: val_acc: 0.899
epoch 4 of 5: val_acc: 0.907


[I 2022-04-20 04:04:40,743] Trial 15 finished with value: 0.9040833333333333 and parameters: {'init_lr': 0.0005218913501129159, 'gamma': 0.8379707182826234, 'batch_size': 16}. Best is trial 13 with value: 0.9136666666666666.


epoch 5 of 5: val_acc: 0.904
epoch 1 of 5: val_acc: 0.878


[I 2022-04-20 04:05:09,019] Trial 16 pruned. 


epoch 2 of 5: val_acc: 0.883
Study statistics: 
  Number of finished trials:  17
  Number of pruned trials:  7
  Number of complete trials:  10
Best trial:
  Value:  0.9136666666666666
  Params: 
    init_lr: 0.0005805506600453946
    gamma: 0.9299130064982555
    batch_size: 16


In [22]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Using ', device)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-04-20 04:06:03,059] A new study created in memory with name: no-name-3c2423ea-313b-4b2b-ae4b-fe5c834b284a


Using  cuda
epoch 1 of 5: val_acc: 0.806
epoch 2 of 5: val_acc: 0.840
epoch 3 of 5: val_acc: 0.857
epoch 4 of 5: val_acc: 0.864


[I 2022-04-20 04:06:30,575] Trial 0 finished with value: 0.8686003989361702 and parameters: {'init_lr': 7.844292368759098e-05, 'gamma': 0.7793832683148582, 'batch_size': 128}. Best is trial 0 with value: 0.8686003989361702.


epoch 5 of 5: val_acc: 0.869
epoch 1 of 5: val_acc: 0.867
epoch 2 of 5: val_acc: 0.891
epoch 3 of 5: val_acc: 0.888
epoch 4 of 5: val_acc: 0.895


[I 2022-04-20 04:07:04,338] Trial 1 finished with value: 0.9031748670212766 and parameters: {'init_lr': 0.005537760222119039, 'gamma': 0.8160780692017976, 'batch_size': 64}. Best is trial 1 with value: 0.9031748670212766.


...

epoch 1 of 5: val_acc: 0.869


[I 2022-04-20 04:30:32,712] Trial 98 pruned. 


epoch 1 of 5: val_acc: 0.758


[I 2022-04-20 04:30:41,837] Trial 99 pruned. 


epoch 1 of 5: val_acc: 0.873
Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  90
  Number of complete trials:  10
Best trial:
  Value:  0.91525
  Params: 
    init_lr: 0.001380615733475873
    gamma: 0.6750421181937485
    batch_size: 32
